In [ ]:
from collections import Counter
import glob
import numpy as np
import sklearn
import pandas as pd

!source /home/admin/cdf38_0-dist/bin/definitions.B
from spacepy import pycdf

In [ ]:
file_paths = glob.glob('/mnt/efs/dasilva/compression-cfha/data/mms_data/**/*.cdf')
file_paths.sort()
file_paths
file_paths[:5]

In [ ]:
phases = [f.split('/')[-2] for f in file_paths]
phases[:5]

In [ ]:
from joblib import delayed, Parallel

def get_length(file_name):
    try:
        cdf = pycdf.CDF(file_name)
        length = cdf['Epoch'].shape[0]
        cdf.close()
    except:
          return 0
    return length


tasks = []
for file_path in file_paths:
    tasks.append(delayed(get_length)(file_path))
    
lengths = Parallel(n_jobs=50, prefer='processes', verbose=10)(tasks)
lengths[:5]

lengths = [get_length(file_name) for file_name in bar(file_paths)]

In [ ]:
df_all = pd.DataFrame({'phase': phases,
                       'file_path': file_paths,
                       'lengths': lengths})
df_all.head()

In [ ]:
SPLIT_FRAC = 0.9
dfs_by_phase = []

for phase in sorted(set(phases)):
    df_phase = df_all[df_all.phase==phase]
    num_rows = len(df_phase.index)
    df_phase_shuffled = df_phase.sample(frac=1)
    df_phase_shuffled['test_train'] = 'train'
    df_phase_shuffled['test_train'].iloc[int(SPLIT_FRAC*num_rows):] = 'test'
    dfs_by_phase.append(df_phase_shuffled)
        
df_all_with_test_train = pd.concat(dfs_by_phase).sort_index()
df_all_with_test_train

In [ ]:
Counter(df_all_with_test_train.test_train)

In [ ]:
df_all_with_test_train.to_csv('/mnt/efs/dasilva/compression-cfha/data/test_train_split.csv', index=False)